<a href="https://colab.research.google.com/github/kroniz-utab/Webinar-Upgrade-Skill-Intro-to-ML/blob/master/Temp_ARIMA_vs_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pmdarima
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse
import warnings
warnings.filterwarnings("ignore")

from google.colab import files


In [ ]:
uploaded = files.upload()

In [ ]:
df = pd.read_csv('temp-and-rain-bangladesh.csv')
df.head()

In [ ]:
df['Date'] = df['Year'].astype(str) + '-' + df['Month'].astype(str) 
df.head()

In [ ]:
df = pd.DataFrame({'date':df['Date'],'temp':df['tem']})
df.head()

In [ ]:
df.tail()

In [ ]:
# Get Data Last 10 Years
time = 12*10
df = df[len(df)-time:]
df

In [ ]:
df.date = pd.to_datetime(df.date)

df.head()

In [ ]:
df = df.set_index("date")
df.head()

In [ ]:
df.info()

In [ ]:
df.index.freq = 'MS'

ax = df['temp'].plot(figsize = (16,5), title = "Monthly Temperature Average")
ax.set(xlabel='Dates',ylabel='Temp Average')

In [ ]:
sea = seasonal_decompose(df['temp'], model='add')
sea.plot();

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (16,7))
sea.seasonal.plot()

In [ ]:
auto_arima(df['temp'], seasonal=True, m=12, max_p=7, max_d=5, max_q=7,
           max_P=4, max_D=4, max_Q=4).summary()

In [ ]:
train_data = df[:len(df)-12]
test_data = df[len(df)-12:]

model = SARIMAX(train_data['temp'],
                order=(2,0,1),
                seasonal_order=(1,1,1,12))
model_result = model.fit()
model_result.summary()

In [ ]:
model_pred = model_result.predict(start= len(train_data),
                                  end = len(df)-1,
                                  typ='levels').rename('Temperature Prediction')
model_pred

In [ ]:
test_data['temp'].plot(figsize = (16,5), legend=True)
model_pred.plot(legend=True)

In [ ]:
test_data['ARIMA_Pred'] = model_pred
test_data

In [ ]:
arima_rmse_error = rmse(test_data['temp'], model_pred)
arima_mse_error = arima_rmse_error**2
mean_value = df['temp'].mean()
print(f'MSE Error: {arima_mse_error}\nRMSE Error: {arima_rmse_error}\nMean:{mean_value}')

In [ ]:
# LSTM
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(train_data)
scaled_train_data = scaler.transform(train_data)
scaled_test_data = scaler.transform(test_data)

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

n_input = 12
n_features = 1
generator = TimeseriesGenerator(scaled_train_data, 
                                scaled_train_data, 
                                length=n_input,
                                batch_size=1)

from keras.models import Sequential
from keras.layers import Dense, LSTM

lstm_model = Sequential()
lstm_model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

lstm_model.summary()

In [ ]:
lstm_model.fit_generator(generator,epochs=20)

In [ ]:
losses_lstm = lstm_model.history.history['loss']
plt.figure(figsize=(14,5))
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(np.arange(0,21,1))
plt.plot(range(len(losses_lstm)), losses_lstm)

In [ ]:
lstm_predictions_scaled = list()

batch = scaled_train_data[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(len(test_data)):
  lstm_pred = lstm_model.predict(current_batch)[0]
  lstm_predictions_scaled.append(lstm_pred)
  current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)

In [ ]:
lstm_predictions_scaled

In [ ]:
lstm_prediction = scaler.inverse_transform(lstm_predictions_scaled)

lstm_prediction

In [ ]:
test_data['LSTM_Predictions'] = lstm_prediction

test_data

In [ ]:
test_data['temp'].plot(figsize = (16,5), legend=True)
test_data['LSTM_Predictions'].plot(legend=True)

In [ ]:
lstm_rmse_error = rmse(test_data['temp'],test_data['LSTM_Predictions'])
lstm_mse_error = lstm_rmse_error**2
mean_value = df['temp'].mean()
print(f'MSE Error: {lstm_mse_error}\nRMSE Error: {lstm_rmse_error}\nMean:{mean_value}')

In [ ]:
rmse_error = [arima_rmse_error, lstm_rmse_error]
mse_error = [arima_mse_error, lstm_mse_error]

errors = pd.DataFrame({'Model':['ARIMA','LSTM'],'RMSE Errors': rmse_error,'MSE Errors': mse_error})
errors

In [ ]:
future_pred = model_result.predict(start= len(df),
                                   end = len(df)+12,
                                   typ='levels').rename('Temperature Future Prediction')
future_pred